<a href="https://colab.research.google.com/github/rodrigodemend/Diagnostico-de-Dados-Clinicos-com-Machine-Learning/blob/main/Notebooks/Importa%C3%A7%C3%A3o_e_Limpeza_dos_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [204]:
import pandas as pd
import numpy as np

In [10]:
dados = pd.read_csv('https://raw.githubusercontent.com/rodrigodemend/Diagnostico-de-Dados-Clinicos-com-Machine-Learning/main/Data/Raw/Kaggle_Sirio_Libanes_ICU_Prediction.csv',
                    sep =';')
dados.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"0,086419753","-0,230769231","-0,283018868","-0,593220339","-0,285714286","0,736842105","0,086419753","-0,230769231","-0,283018868","-0,586206897","-0,285714286","0,736842105","0,237113402",0,"-0,162393162","-0,5","0,208791209","0,898989899","-0,247863248","-0,459459459","-0,432835821","-0,636363636","-0,420289855","0,736842105",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"0,333333333","-0,230769231","-0,132075472","-0,593220339","0,535714286","0,578947368","0,333333333","-0,230769231","-0,132075472","-0,586206897","0,535714286","0,578947368","0,443298969",0,"-0,025641026","-0,5","0,714285714","0,838383838","-0,076923077","-0,459459459","-0,313432836","-0,636363636","0,246376812","0,578947368",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,938949939","-0,938949939",...,"-0,994911922",-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"-0,107142857","0,736842105",NaN,NaN,NaN,NaN,"-0,107142857","0,736842105",NaN,NaN,NaN,NaN,"0,318681319","0,898989899",NaN,NaN,NaN,NaN,"-0,275362319","0,736842105",NaN,NaN,NaN,NaN,-1,-1,NaN,NaN,NaN,NaN,-1,-1,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0,0,0,-1.0,"-0,871657754","-0,871657754","-0,871657754","-0,871657754",-1.0,"-0,863874346","-0,863874346","-0,863874346","-0,863874346",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,414634146","-0,414634146","-0,414634146","-0,414634146",-1.0,"-0,97906855","-0,97906855",...,"-0,996762132",-1.0,"-0,243021346","-0,338537325","-0,213030951","-0,31785923","0,033779029","0,665932072","-0,283950617","-0,376923077","-0,188679245","-0,379310345","0,035714286","0,6315

In [135]:
dados.describe()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_DIFF,BE_ARTERIAL_DIFF,BE_VENOUS_DIFF,BIC_ARTERIAL_DIFF,BIC_VENOUS_DIFF,BILLIRUBIN_DIFF,BLAST_DIFF,CALCIUM_DIFF,CREATININ_DIFF,FFA_DIFF,GGT_DIFF,GLUCOSE_DIFF,HEMATOCRITE_DIFF,HEMOGLOBIN_DIFF,INR_DIFF,LACTATE_DIFF,LEUKOCYTES_DIFF,LINFOCITOS_DIFF,NEUTROPHILES_DIFF,P02_ARTERIAL_DIFF,P02_VENOUS_DIFF,PC02_ARTERIAL_DIFF,PC02_VENOUS_DIFF,PCR_DIFF,PH_ARTERIAL_DIFF,PH_VENOUS_DIFF,PLATELETS_DIFF,POTASSIUM_DIFF,SAT02_ARTERIAL_DIFF,SAT02_VENOUS_DIFF,SODIUM_DIFF,TGO_DIFF,TGP_DIFF,TTPA_DIFF,UREA_DIFF,DIMER_DIFF,ICU
count,1925.000000,1925.000000,1925.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,821.0,1925.000000
mean,192.000000,0.467532,0.368831,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,0.213021,0.158333,0.809896,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.267532
std,111.168431,0.499074,0.482613,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,0.409549,0.365148,0.392485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.442787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000
25%,96.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000
50%,192.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000
75%,288.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000
max,384.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000


## Eliminando os valores faltantes

In [185]:
def removendo_pacientes_com_vazamento_de_dados(dados):
  # Criando um set que irá servir para selecionar os pacientes para remover
  pacientes_para_remover = set()

  # Rodando um for para cada paciente
  for paciente, dados_do_paciente in dados.groupby('PATIENT_VISIT_IDENTIFIER', as_index=False):
    
    # Rodando um for para cada coluna 
    # Aqui vamos achar os pacientes onde o primeiro valor válido naquela coluna já estão na ICU
    # e para que não tenha vazamento de dados, iremos adicionar o paciente no set para removê-los
    for coluna in dados_do_paciente.columns:

      # Pulando a coluna ICU pois não queremos analisa-la
      if coluna != 'ICU':

        # Selecionando apenas a coluna que estamos analisando e a coluna ICU
        # dropna irá remover todas as linhas onde existem valores nulos, porém como só pode
        # existir valores nulos na coluna que estamos analisando, a primeira linha que irá retornar
        # será o primeiro valor válido daquela coluna
        dados_sem_NaN = dados_do_paciente[[coluna, 'ICU']].dropna().reset_index(drop=True)
        
        # Caso exista algum valor válido para a coluna, irá entrar no if
        if dados_sem_NaN.shape[0] > 0:

          # Verificando se o paciente está na ICU no primeiro valor válido para a coluna
          if dados_sem_NaN.iloc[0]['ICU'] == 1:
            
            # Adicionando o paciente para poder removê-lo
            pacientes_para_remover.add(paciente)
        
        # Caso o paciente tenha a coluna inteira nula, irá para o else
        else:
          
          # Adicionando o paciente para poder removê-lo
          pacientes_para_remover.add(paciente)

  # Retornando apenas os dados sem os pacientes que selecionamos para remover
  return dados[~dados['PATIENT_VISIT_IDENTIFIER'].isin(pacientes_para_remover)]

Removendo os pacientes com vazamento de dados no bfill

In [189]:
dados_sem_pacientes_com_vazamento_de_dados = removendo_pacientes_com_vazamento_de_dados(dados)

Aplicando o ffill e o bfill para preencher os valores faltantes

In [197]:
# Criando uma coluna que irá se perder no agrupamento para preencher os valores faltantes
dados_sem_pacientes_com_vazamento_de_dados['PATIENT_VISIT_IDENTIFIER_2'] = dados_sem_pacientes_com_vazamento_de_dados['PATIENT_VISIT_IDENTIFIER']

# Realizando o agrupamento e preenchimento dos valores faltantes
dados_sem_valores_faltantes = dados_sem_pacientes_com_vazamento_de_dados.groupby('PATIENT_VISIT_IDENTIFIER_2', as_index=False).fillna(method='ffill').fillna(method='bfill')

# Visualizando os dados sem valores faltantes
dados_sem_valores_faltantes.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,938949939","-0,938949939",...,"-0,994911922",-1.0,"0,086419753","-0,230769231","-0,283018868","-0,593220339","-0,285714286","0,736842105","0,086419753","-0,230769231","-0,283018868","-0,586206897","-0,285714286","0,736842105","0,237113402",0,"-0,162393162","-0,5","0,208791209","0,898989899","-0,247863248","-0,459459459","-0,432835821","-0,636363636","-0,420289855","0,736842105",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,938949939","-0,938949939",...,"-0,994911922",-1.0,"0,333333333","-0,230769231","-0,132075472","-0,593220339","0,535714286","0,578947368","0,333333333","-0,230769231","-0,132075472","-0,586206897","0,535714286","0,578947368","0,443298969",0,"-0,025641026","-0,5","0,714285714","0,838383838","-0,076923077","-0,459459459","-0,313432836","-0,636363636","0,246376812","0,578947368",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,938949939","-0,938949939",...,"-0,994911922",-1.0,"0,333333333","-0,230769231","-0,132075472","-0,593220339","0,535714286","0,578947368","0,333333333","-0,230769231","-0,132075472","-0,586206897","0,535714286","0,578947368","0,443298969",0,"-0,025641026","-0,5","0,714285714","0,838383838","-0,076923077","-0,459459459","-0,313432836","-0,636363636","0,246376812","0,578947368",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,938949939","-0,938949939",...,"-0,994911922",-1.0,"0,333333333","-0,230769231","-0,132075472","-0,593220339","-0,107142857","

In [198]:
# Visualizando que não existem mais valores faltantes
dados_sem_valores_faltantes.isnull().sum().sum()

0

## Transformando os dados

Aqui iremos deixar o conjunto de dados no formato em que só teremos os exames da primeira janela. Na coluna ICU será representado caso o paciente tenha ido para a UTI, porém não importa em qual momento.

Aqui iremos utilizar apenas a janela 0-2 para prever.

In [239]:
def transforma_dados(dados_do_paciente):
  if np.any(dados_do_paciente['ICU']):
    dados_do_paciente.loc[dados_do_paciente['WINDOW'] == '0-2', 'ICU'] = 1
  df = dados_do_paciente.loc[dados_do_paciente['WINDOW'] == '0-2']
  return df.drop(['PATIENT_VISIT_IDENTIFIER', 'WINDOW'], axis=1)

dados_limpos = dados_sem_valores_faltantes.groupby('PATIENT_VISIT_IDENTIFIER').apply(transforma_dados).reset_index(drop=True)
dados_limpos.head()

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,...,DIMER_MIN,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,938949939","-0,938949939","-0,938949939",...,"-0,994911922","-0,994911922",-1.0,"0,086419753","-0,230769231","-0,283018868","-0,593220339","-0,285714286","0,736842105","0,086419753","-0,230769231","-0,283018868","-0,586206897","-0,285714286","0,736842105","0,237113402",0,"-0,162393162","-0,5","0,208791209","0,898989899","-0,247863248","-0,459459459","-0,432835821","-0,636363636","-0,420289855","0,736842105",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
1,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,938949939","-0,938949939","-0,938949939",...,"-0,978028755","-0,978028755",-1.0,"-0,489711934","-0,685470085","-0,048218029","-0,645951036","0,357142857","0,935672515","-0,50617284","-0,815384615","-0,056603774","-0,517241379","0,357142857","0,947368421","-0,525773196","-0,5125","-0,111111111","-0,714285714","0,604395604","0,95959596","-0,435897436","-0,491891892",0,"-0,575757576","0,101449275",1,"-0,547826087","-0,533742331","-0,603053435","-0,764705882",-1,"-0,95959596","-0,51552795","-0,351327692","-0,747001091","-0,756272401",-1,"-0,961262106",1
2,0,40th,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,"-0,263157895","-0,263157895","-0,263157895","-0,263157895",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.0,"-0,972789116","-0,972789116","-0,972789116",...,"-0,978028755","-0,978028755",-1.0,"0,012345679","-0,369230769","-0,528301887","-0,457627119","-0,285714286","0,684210526","0,012345679","-0,369230769","-0,528301887","-0,448275862","-0,285714286","0,684210526","0,175257732","-0,1125","-0,384615385","-0,357142857","0,208791209","0,878787879","-0,299145299","-0,556756757","-0,626865672","-0,515151515","-0,420289855","0,684210526",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
3,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"0,605263158","0,605263158","0,605263158","0,605263158",-1.0,-1,-1,-1,-1,-1.0,-1,-1,-1,-1,-1.0,"-0,317073171","-0,317073171","-0,317073171","-0,317073171",-1.

In [240]:
# Salvando dados limpos e prontos para criação do modelo
dados_limpos.to_csv('/content/cleaned_data.csv', encoding='utf-8', index=True)